# Step 2 Data Wrangling

**Introduction**

IEEE Computational Intelligence Society (IEEE-CIS) works across a variety of AI and machine learning areas, including deep neural networks, fuzzy systems, evolutionary computation, and swarm intelligence. Today they’re partnering with Vesta, the world’s leading payment service company, seeking the best solutions for the fraud detection industry. The fraud prevention system used by Vesta is actually saving consumers millions of dollars per year. Researchers from the IEEE-CIS want to improve fraud detection accuracy but also the customer experiences.

**Data Source**

The data comes from Vesta’s real-world e-commerce transactions and contains a wide range of features from device type to product features, available in Kaggle competition (https://www.kaggle.com/c/ieee-fraud-detection/data). Only train_identity and train_transaction datasets will be used for this project.

**The Data Science Method**  


1.   Problem Identification 

2.   **Data Wrangling** 
 
  * Data Collection: loading and joining
  * Data Definition
      - Column names
      - Data types
      - Count/Percent of unique values
  * Data Cleaning
      - NA or missing data
      - Duplicates
 
3.   Exploratory Data Analysis 

4.   Pre-processing,Training Data Development and Modeling 

5.   Documentation

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**2.1 Data Collection**

*Data Loading and Organization*

In [2]:
%%time
train_transaction = pd.read_csv('.../data/train_transaction.csv')
train_identity = pd.read_csv('.../data/train_identity.csv')

CPU times: user 18.6 s, sys: 6.58 s, total: 25.2 s
Wall time: 26.2 s


**2.2 Data Definitions**

##### Transaction Tables:
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

Categorical Features: ProductCD, card1 - card6, addr1, addr2, Pemaildomain Remaildomain, M1 - M9

##### Identity Tables
Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions. 
They're collected by Vesta’s fraud protection system and digital security partners.(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

Categorical Features: DeviceType, DeviceInfo, id12 - id38

In [3]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [5]:
print('train_transaction shape {}'.format(train_transaction.shape))
print('train_identity shape {}'.format(train_identity.shape))

train_transaction shape (590540, 394)
train_identity shape (144233, 41)


In [6]:
# check if all transactions have associated identity 
print(np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique())))

144233


We have a binary classification problem with imbalance data. 
We combine the transaction and identity data based on the TransactionID. Note that not all transactions have associated identity information.

In [7]:
# combine transaction and identity data
data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [8]:
# columns with only one unique value
one_value = [col for col in data.columns if data[col].nunique() <= 1]
print(len(one_value))

0


**2.2 Data Definition**

In [9]:
print('training set shape {}'.format(data.shape))

training set shape (590540, 434)


In [10]:
# data infos
data.info

<bound method DataFrame.info of         TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  \
0             2987000        0          86400           68.50         W   
1             2987001        0          86401           29.00         W   
2             2987002        0          86469           59.00         W   
3             2987003        0          86499           50.00         W   
4             2987004        0          86506           50.00         H   
...               ...      ...            ...             ...       ...   
590535        3577535        0       15811047           49.00         W   
590536        3577536        0       15811049           39.50         W   
590537        3577537        0       15811079           30.95         W   
590538        3577538        0       15811088          117.00         W   
590539        3577539        0       15811131          279.95         W   

        card1  card2  card3       card4  card5  ...                

**Unique Values**

In [11]:
# percentage of transactions that have associated identity
print(144233/590540)

0.2442391709283029


- 24.4% of transactions in the data set have an associated identity

**Look for duplicates**

In [12]:
print(train_transaction[train_transaction.duplicated()])
print(train_identity[train_identity.duplicated()])

Empty DataFrame
Columns: [TransactionID, isFraud, TransactionDT, TransactionAmt, ProductCD, card1, card2, card3, card4, card5, card6, addr1, addr2, dist1, dist2, P_emaildomain, R_emaildomain, C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, D1, D2, D3, D4, D5, D6, D7, D8, D9, D10, D11, D12, D13, D14, D15, M1, M2, M3, M4, M5, M6, M7, M8, M9, V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V26, V27, V28, V29, V30, V31, V32, V33, V34, V35, V36, V37, V38, V39, V40, V41, V42, V43, V44, V45, ...]
Index: []

[0 rows x 394 columns]
Empty DataFrame
Columns: [TransactionID, id_01, id_02, id_03, id_04, id_05, id_06, id_07, id_08, id_09, id_10, id_11, id_12, id_13, id_14, id_15, id_16, id_17, id_18, id_19, id_20, id_21, id_22, id_23, id_24, id_25, id_26, id_27, id_28, id_29, id_30, id_31, id_32, id_33, id_34, id_35, id_36, id_37, id_38, DeviceType, DeviceInfo]
Index: []

[0 rows x 41 columns]


**Check for missing values**

In [13]:
print('There are {} columns in training set with missing values.'.format(data.isnull().any().sum()))

There are 414 columns in training set with missing values.


In [14]:
# percent of each feature that is missing
nas = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)/len(data),columns = ['percent'])
nas[nas['percent'] > 0]

,percent
id_24,0.991962
id_25,0.991310
id_07,0.991271
id_08,0.991271
id_21,0.991264
...,...
V309,0.000020
V308,0.000020
V307,0.000020
V306,0.000020


**2.3 Data Cleaning**

We will drop columns with more than 60% of missing values, columns with only one unique value (all missing)

In [15]:
# filter out features that have more than 60% of missing data
data_new = nas['percent'] < 0.4
nas[data_new]

,percent
M6,0.286788
V42,0.286126
V52,0.286126
V51,0.286126
V50,0.286126
...,...
C11,0.000000
C12,0.000000
C13,0.000000
C14,0.000000


In [16]:
df = data.loc[:, data_new]
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
print(df.columns.tolist())

['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D4', 'D10', 'D15', 'M6', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', '

**Card Features**

In [18]:
df[['TransactionDT', 'TransactionAmt']].describe().T

,count,mean,std,min,25%,50%,75%,max
TransactionDT,590540.0,7.372311e+06,4.617224e+06,86400.000,3027057.750,7306527.500,11246620.0,1.581113e+07
TransactionAmt,590540.0,1.350272e+02,2.391625e+02,0.251,43.321,68.769,125.0,3.193739e+04


In [19]:
df[['card1', 'card2', 'card3', 'card4', 'card5', 'card6']].describe().T

,count,mean,std,min,25%,50%,75%,max
card1,590540.0,9898.734658,4901.170153,1000.0,6019.0,9678.0,14184.0,18396.0
card2,581607.0,362.555488,157.793246,100.0,214.0,361.0,512.0,600.0
card3,588975.0,153.194925,11.336444,100.0,150.0,150.0,150.0,231.0
card5,586281.0,199.278897,41.244453,100.0,166.0,226.0,226.0,237.0


In [20]:
df[['card4', 'card6']].head()

,card4,card6
0,discover,credit
1,mastercard,credit
2,visa,debit
3,mastercard,debit
4,mastercard,credit


We can see that card4 and card6 represent the payment methods

In [21]:
df[['addr1', 'addr1', 'P_emaildomain']].head()

,addr1,addr1,P_emaildomain
0,315.0,315.0,NaN
1,325.0,325.0,gmail.com
2,330.0,330.0,outlook.com
3,476.0,476.0,yahoo.com
4,420.0,420.0,gmail.com


In [22]:
df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']].describe().T

,count,mean,std,min,25%,50%,75%,max
C1,590540.0,14.092458,133.569018,0.0,1.0,1.0,3.0,4685.0
C2,590540.0,15.269734,154.668899,0.0,1.0,1.0,3.0,5691.0
C3,590540.0,0.005644,0.150536,0.0,0.0,0.0,0.0,26.0
C4,590540.0,4.092185,68.848459,0.0,0.0,0.0,0.0,2253.0
C5,590540.0,5.571526,25.786976,0.0,0.0,0.0,1.0,349.0
C6,590540.0,9.071082,71.508467,0.0,1.0,1.0,2.0,2253.0
C7,590540.0,2.848478,61.727304,0.0,0.0,0.0,0.0,2255.0
C8,590540.0,5.144574,95.378574,0.0,0.0,0.0,0.0,3331.0
C9,590540.0,4.480240,16.674897,0.0,0.0,1.0,2.0,210.0
C10,590540.0,5.240343,95.581443,0.0,0.0,0.0,0.0,3257.0


C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked

In [28]:
df[['D1', 'D4', 'D10', 'D15']].describe().T

,count,mean,std,min,25%,50%,75%,max
D1,589271.0,94.347568,157.660387,0.0,0.0,3.0,122.0,640.0
D4,421618.0,140.002441,191.096774,-122.0,0.0,26.0,253.0,869.0
D10,514518.0,123.982137,182.615225,0.0,0.0,15.0,197.0,876.0
D15,501427.0,163.744579,202.726660,-83.0,0.0,52.0,314.0,879.0


In [24]:
df['M6'].describe()

count     421180
unique         2
top            F
freq      227856
Name: M6, dtype: object

D features: timedelta, such as days between previous transaction, etc.. M feature: match on card information

We will also drop certains rows with more than 10 NAs

In [26]:
df_reduced = df[df.isnull().sum(axis=1) < 10]
df_reduced

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.000000,0.000000,0.000000
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590534,3577534,0,15811030,67.505,C,5812,408.0,185.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,117.0,317.5,669.5,317.5,0.0,2234.0,0.0,0.000000,0.000000,0.000000


We will explore the dataset in next step and deal with the rest of the missing values after visualizing each features in feature engineering step.

**Export data to new file**

In [27]:
df_reduced.to_csv('.../data/step2_output.csv', index=False)